# Using the Collector class to interface with the SQLite tables

In [1]:
import os
import logging
import time

from FastEHR.database import SQLiteDataCollector

logging.basicConfig(level=logging.INFO)

!pwd

%load_ext autoreload
%autoreload 2

/home/ubuntu/Documents/GitHub/SurvivEHR/FastEHR/examples/1_build_database


# Paths to example dataset and location to save generated database

In [2]:
path_to_directory = os.getcwd() + "/../data/"
PATH_TO_DB = os.getcwd() + "/../data/_built/example_database.db"
print(f"Saving databse to {PATH_TO_DB}")

load = False
if load:
    logging.warning(f"Load is true, if you want to re-build database set to False")

Saving databse to /home/ubuntu/Documents/GitHub/SurvivEHR/FastEHR/examples/1_build_database/../data/_built/example_database.db


# Connecting to the SQLite database

In [3]:
collector = SQLiteDataCollector(db_path=PATH_TO_DB)
collector.connect()

# Running open queries

When we built the database, we created different tables

In [4]:
collector.cursor.execute("""SELECT name FROM sqlite_master WHERE type='table';""")   # 
results = collector.cursor.fetchall()
for result in results:
    print(result)

('static_table',)
('diagnosis_table',)
('measurement_25_Hydroxyvitamin_D2_level_92_',)


We can perform open queries on any of the database tables

In [5]:
collector.cursor.execute("""SELECT * FROM static_table WHERE sex=='M' AND imd=='1' LIMIT 10""")   # 
results = collector.cursor.fetchall()
for result in results:
    print(result)

(20960, 1, 'MISSING', '1940-07-15', 'M', 'E', 1, 'South East', '2008-07-23', '2008-07-23', '2022-03-17')
(20961, 2, 'WHITE', '1940-07-21', 'M', 'E', 1, 'North West', '2008-07-29', '2008-07-29', '2022-03-23')
(20962, 3, 'MIXED', '1940-07-27', 'M', 'E', 1, 'South East', '2008-08-04', '2008-08-04', '2022-03-29')
(20963, 4, 'ASIAN', '1940-08-02', 'M', 'E', 1, 'West Midlands', '2008-08-10', '2008-08-10', '2022-04-04')
(20964, 5, 'BLACK', '1940-08-08', 'M', 'E', 1, 'South East', '2008-08-16', '2008-08-16', '2022-04-10')
(20966, 1, 'MISSING', '1940-08-14', 'M', 'E', 1, 'West Midlands', '2008-08-22', '2008-08-22', '2022-04-16')
(20967, 2, 'WHITE', '1940-08-20', 'M', 'E', 1, 'South East', '2008-08-28', '2008-08-28', '2022-04-22')
(20968, 3, 'MIXED', '1940-08-26', 'M', 'E', 1, 'West Midlands', '2008-09-03', '2008-09-03', '2022-04-28')
(20969, 4, 'ASIAN', '1940-09-01', 'M', 'E', 1, 'South East', '2008-09-09', '2008-09-09', '2022-05-04')
(20970, 5, 'BLACK', '1940-09-07', 'M', 'E', 1, 'West Midland

In [6]:
collector.cursor.execute("""SELECT * FROM diagnosis_table WHERE event=='AF' LIMIT 10""")   # measurement_ACE_Inhibitors_D2T
results = collector.cursor.fetchall()
for result in results:
    print(result)

(20960, 4, 'AF', '1992-06-12')
(20965, 4, 'AF', '1992-06-13')
(20968, 4, 'AF', '1992-06-14')
(20972, 1, 'AF', '1992-06-15')
(20974, 5, 'AF', '1992-06-16')
(20979, 1, 'AF', '1992-06-17')


TODO: measurement example tables dont contain data, so no tables were built

In [7]:
collector.cursor.execute("""SELECT * FROM measurement_25_Hydroxyvitamin_D2_level_92_ LIMIT 10""")   # measurement_ACE_Inhibitors_D2T
results = collector.cursor.fetchall()
for result in results:
    print(result)

(20960, 1, '25_Hydroxyvitamin_D2_level_92_', None, ' 2001-08-01')
(20960, 1, '25_Hydroxyvitamin_D2_level_92_', 10.0, ' 2001-08-02')
(20960, 1, '25_Hydroxyvitamin_D2_level_92_', None, ' 2001-08-03')
(20960, 2, '25_Hydroxyvitamin_D2_level_92_', None, ' 2001-08-15')


# TODO: add examples on all of the other collector functionality

These methods are used internally when building a Dataset

## Extracting distinct values

For example, this can be patient identifiers to determine chunking

In [8]:
collector._extract_distinct(["static_table", "diagnosis_table"], "PRACTICE_ID")[:5]

[20960, 20961, 20962, 20963, 20964]

# Collecting meta information

In [9]:
meta_information = collector.get_meta_information()
display(meta_information)

INFO:root:

INFO:root:	 Static meta information
INFO:root:	 Diagnosis meta information
INFO:root:	 Measurements meta information
                                      Measurements: 100%|█| 1/1 [00:00<00:00, 14


{'static_table': {'SEX':   category  count
  0        F     33
  1        I     33
  2        M     34,
  'IMD':    category  count
  0       NaN     16
  1       1.0     17
  2       2.0     17
  3       3.0     17
  4       4.0     17
  5       5.0     16,
  'ETHNICITY':   category  count
  0    ASIAN     20
  1    BLACK     20
  2  MISSING     20
  3    MIXED     20
  4    WHITE     20},
 'diagnosis_table':                                   event  count
 0                                    AF      6
 1                     ASTHMA_PUSHASTHMA      1
 2           CHRONICFATIGUESYNDROMEMM_V2      1
 3                                 DEATH    100
 4                   HAEMOCHROMATOSIS_V2      1
 5                          HYPERTENSION      1
 6                LYMPHOMA_PREVALENCE_V2      1
 7                          OSTEOPOROSIS      1
 8   OTHER_CHRONIC_LIVER_DISEASE_OPTIMAL      1
 9                       STROKE_HAEMRGIC      6
 10                        TYPE2DIABETES      2,
 'measurem

In [10]:
collector.disconnect()

# Below needs to be moved to Dataset example folder

In [11]:
PATH_TO_DS = path_to_directory + "dataset/"

# Build 
dm = FoundationalDataModule(path_to_db=PATH_TO_DB,
                            path_to_ds=PATH_TO_DS,
                            load=True,
                            include_diagnoses=True,                            
                            include_measurements=True,
                            drop_missing_data=False,
                            drop_empty_dynamic=False,        # Change to True in real use case (False due to lack of example data)
                            tokenizer="tabular",
                            # practice_inclusion_conditions=["COUNTRY = 'E'"],
                            num_threads=10
                           )

vocab_size = dm.train_set.tokenizer.vocab_size

print(f"{len(dm.train_set)} training patients")
print(f"{len(dm.val_set)} validation patients")
print(f"{len(dm.test_set)} test patients")
print(f"{vocab_size} vocab elements")

NameError: name 'FoundationalDataModule' is not defined

# Meta information

In building the dataset, summary statistics of the events were accumulated and stored in meta_information.

These were stored on file, but are also loaded into the data module to be used for
* pre-processing
* tokenizer building

In [ ]:
for key in dm.train_set.meta_information.keys():
    print(f"\n\n{key}\n" + "="*len(key) + "\n")
    display(dm.train_set.meta_information[key])

There are also stored within the tokenizer

In [ ]:
import polars as pl
pl.Config.set_tbl_rows(300)
pl.Config.set_fmt_str_lengths(100)
print(dm.train_set.tokenizer._event_counts)

# Test data loading times (so we can optimise cpu usage)

In [ ]:
import pyarrow.parquet as pq
import time

print(PATH_TO_DS + "split=train/")
dataset1 = pq.ParquetDataset(PATH_TO_DS + "split=train/", 
                             filters=[('PRACTICE_ID','=','20968')]
                             )

# Time to read 
start = time.time()   # starting time
df  = dataset1.read().to_pandas()
print(df[df["row_nr"] == df.row_nr[0]])
print(time.time() - start)


## Time to load individual samples

TODO: this error is due to a poorly populated example dataset 

In [ ]:
from tqdm import tqdm
import numpy as np

times = []
start = time.time()   # starting time
for row_idx, row in enumerate(tqdm(dm.train_set)):
    # print(f"Sample loaded in {time.time()-start} seconds")
    times.append(time.time()-start)
    start = time.time()
    if row_idx > 100:
        break

print(np.mean(times))

## Time to load batch (with only one worker)

In [ ]:
# for batch in generator:
#     print(batch.columns)
#     break

In [ ]:
times = []
start = time.time()   # starting time
for batch_idx, batch in enumerate(tqdm(dm.train_dataloader())):
    # print(f"batch loaded in {time.time()-start} seconds")    
    times.append(time.time()-start)
    start = time.time()
    if batch_idx > 2:
        break
print(np.mean(times))

# for key in batch.keys():
#     print(f"{key}".ljust(20) + f"{batch[key].shape}")

# tokens = batch["tokens"][0].tolist()    
# sentence = dm.decode(tokens).split(" ")
# for token, value in zip(sentence, batch["values"][0].tolist()):
#     print(f"{token}:".ljust(40) + f"{value}")

In [ ]:
dm.train_set.view_sample(2, max_dynamic_events=12, report_time=True)

# Tokenizer keys

Here we can see that we have built a tokenizer on only the few event examples given in the example dataset. 

In [ ]:
display(dm.train_set.tokenizer._stoi.keys())